# EDA: Shopping Behavior Updated
* Practice pivot_table, pd.crosstab, groupby, stack, and unstack for data aggregation and analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Google Drive: Mounting Google Drive locally

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read from
path = '/content/drive/MyDrive/Colab/Datasets/shopping_behavior_updated.csv'
df = pd.read_csv(path)
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

## EDA/ Discovery Questions
1. Identify the top selling categories and Items
2. Assess Sales by Season, % of Sales by Season
3. Customer Analysis
  - Distribution of Age by Gender, qcut
  - Distribution of Rating by Category
  - Distribution of Frequency of Purchase and Previous Payments

## Code Samples

In [ ]:
# df.groupby('Gender').agg(
#     avg_satisfaction=('Customer_Satisfaction', 'mean'),
#     min_satisfaction=('Customer_Satisfaction', 'min'),
#     max_satisfaction=('Customer_Satisfaction', 'max'),
#     )

# pd.cut(df['Age'], bins=5).unique()

# pd.qcut(df['Age'], q=[0, .25, .5, .75, 1.],
#         labels = ['very young', 'young', 'middle age', 'old'])

# pd.crosstab(df['Gender'], df['Marital_Status'], margins=True, normalize='columns')

# df.pivot_table(index=['Marital_Status'], values=['Frequency_of_Purchase', 'Customer_Satisfaction'], aggfunc=['mean', 'sum'], margins=True)

# pivot_example = df.pivot_table(index=['Gender', 'Marital_Status'], values=['Frequency_of_Purchase', 'Customer_Satisfaction'], aggfunc='median')

# df.pivot_table(index=['Marital_Status'], columns= df['Time_of_Purchase'].dt.month,values= ['Customer_Satisfaction'], aggfunc='mean')

In [ ]:
# rename columns
df.rename(columns={'Purchase Amount (USD)': 'Purchase Amount'}, inplace=True)

In [ ]:
# group by single column with multiple aggregations
df_category = df.groupby('Category').agg(
    cat_count=('Category', 'count'),
    purchase_sum=('Purchase Amount', 'sum'),
    purchase_mean=('Purchase Amount', 'mean'),
    purchase_med=('Purchase Amount', 'median'),
    )

# calculate percent of total
df_category['cat_count_perc'] = df_category['cat_count'] / df_category['cat_count'].sum()

# reorder columns
order = ['cat_count', 'cat_count_perc','purchase_sum', 'purchase_mean', 'purchase_med']
df_category = df_category[order]
df_category

,cat_count,cat_count_perc,purchase_sum,purchase_mean,purchase_med
Category,,,,,
Accessories,1240,0.317949,74200,59.838710,60.0
Clothing,1737,0.445385,104264,60.025331,60.0
Footwear,599,0.153590,36093,60.255426,60.0
Outerwear,324,0.083077,18524,57.172840,54.5


In [ ]:
# pivot table with 1 index and 1 column
df.pivot_table(index=['Category'], columns=['Season'], values=['Purchase Amount'], aggfunc=['sum'])

sum                     
            Purchase Amount                     
Season                 Fall Spring Summer Winter
Category                                        
Accessories           19874  17007  19028  18291
Clothing              26220  27692  23078  27274
Footwear               8665   9555   9393   8480
Outerwear              5259   4425   4278   4562

In [ ]:
# pivot table with 1 index and 1 column
df_season_cat_sales = df.pivot_table(index=['Category'], columns=['Season'], values=['Purchase Amount'], aggfunc=['sum'])
df_season_cat_sales

sum                     
            Purchase Amount                     
Season                 Fall Spring Summer Winter
Category                                        
Accessories           19874  17007  19028  18291
Clothing              26220  27692  23078  27274
Footwear               8665   9555   9393   8480
Outerwear              5259   4425   4278   4562

In [ ]:
# calculate column totals
column_totals = df_season_cat_sales.sum(axis=0)

# calculate perc of sales from pivot table output
df_season_cat_perc = df_season_cat_sales.div(column_totals, axis=1)
df_season_cat_perc

sum                              
            Purchase Amount                              
Season                 Fall    Spring    Summer    Winter
Category                                                 
Accessories        0.331134  0.289831  0.341144  0.312096
Clothing           0.436869  0.471924  0.413755  0.465371
Footwear           0.144373  0.162835  0.168403  0.144693
Outerwear          0.087624  0.075410  0.076698  0.077841

In [ ]:
# calculate row totals
row_totals = df_season_cat_sales.sum(axis=1)

# calculate perc of sales from pivot table output
df_season_cat_row_perc = df_season_cat_sales.div(row_totals, axis=0)
df_season_cat_row_perc

sum                              
            Purchase Amount                              
Season                 Fall    Spring    Summer    Winter
Category                                                 
Accessories        0.267844  0.229205  0.256442  0.246509
Clothing           0.251477  0.265595  0.221342  0.261586
Footwear           0.240074  0.264733  0.260244  0.234949
Outerwear          0.283902  0.238879  0.230944  0.246275

In [ ]:
# distribution of Age by Gender
df_gender_age = df.groupby('Gender').agg(
    gender_count=('Gender', 'count'),
    age_min=('Age', 'min'),
    age_25th=('Age', lambda x: x.quantile(0.25)),
    age_mean=('Age', 'mean'),
    age_med=('Age', 'median'),
    age_75th=('Age', lambda x: x.quantile(0.75)),
    age_max=('Age', 'max'),
    age_std=('Age', 'std'),
    age_var=('Age', 'var'),
    age_sem=('Age', 'sem'),
    )

df_gender_age.T

Gender,Female,Male
gender_count,1248.000000,2652.000000
age_min,18.000000,18.000000
age_25th,31.000000,31.000000
age_mean,44.007212,44.097285
age_med,44.000000,44.000000
age_75th,57.000000,57.000000
age_max,70.000000,70.000000
age_std,14.953843,15.328257
age_var,223.617430,234.955451
age_sem,0.423297,0.297650


In [ ]:
for value in df['Gender'].unique():
  print(f'Gender == {value}')
  print(df[df['Gender']== value]['Age'].describe(percentiles=[0.1, 0.15, .25, 0.35, .5, .75, .95, 0.99]))

Gender == Male
count    2652.000000
mean       44.097285
std        15.328257
min        18.000000
10%        23.000000
15%        25.650000
25%        31.000000
35%        36.000000
50%        44.000000
75%        57.000000
95%        68.000000
99%        70.000000
max        70.000000
Name: Age, dtype: float64
Gender == Female
count    1248.000000
mean       44.007212
std        14.953843
min        18.000000
10%        23.000000
15%        26.000000
25%        31.000000
35%        36.000000
50%        44.000000
75%        57.000000
95%        68.000000
99%        70.000000
max        70.000000
Name: Age, dtype: float64


In [ ]:
# create age bin fields
  # Fields: lower_bin, upper_bin, label

df['Age_Bin'] = pd.cut(df['Age'], bins=5)

df['Age_Lower_Bin'] = df['Age_Bin'].apply(lambda x: round(x.left, 1))
df['Age_Upper_Bin'] = df['Age_Bin'].apply(lambda x: round(x.right, 1))

In [ ]:
# create Age_Group label based on quantile bins
df['Age_Group'] = pd.qcut(df['Age'], q=[0, .25, .5, .75, 1.],
        labels = ['age_1', 'age_2', 'age_3', 'age_4'])

df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount,Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,Age_Bin,Age_Lower_Bin,Age_Upper_Bin,Age_Group
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,"(49.2, 59.6]",49.2,59.6,age_3
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,"(17.948, 28.4]",17.9,28.4,age_1
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,"(49.2, 59.6]",49.2,59.6,age_3
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,"(17.948, 28.4]",17.9,28.4,age_1
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,"(38.8, 49.2]",38.8,49.2,age_3


### User Rating Analysis

In [ ]:
df['Review Rating'].describe(percentiles=[0.1, 0.15, .25, 0.35, .5, .65, .75, .95, 0.99])

,Review Rating
count,3900.000000
mean,3.749949
std,0.716223
min,2.500000
10%,2.800000
15%,2.900000
25%,3.100000
35%,3.400000
50%,3.700000
65%,4.100000


In [ ]:
# distribution of Age by Gender
df_cat_review = df.groupby('Category').agg(
gender_count=('Category', 'count'),
review_rating_min=('Review Rating', 'min'),
review_rating_25th=('Review Rating', lambda x: x.quantile(0.25)),
review_rating_mean=('Review Rating', 'mean'),
review_rating_med=('Review Rating', 'median'),
review_rating_75th=('Review Rating', lambda x: x.quantile(0.75)),
review_rating_max=('Review Rating', 'max'),
review_rating_std=('Review Rating', 'std'),
review_rating_var=('Review Rating', 'var'),
review_rating_sem=('Review Rating', 'sem'),
    )

df_cat_review.T

Category,Accessories,Clothing,Footwear,Outerwear
gender_count,1240.000000,1737.000000,599.000000,324.000000
review_rating_min,2.500000,2.500000,2.500000,2.500000
review_rating_25th,3.200000,3.100000,3.200000,3.100000
review_rating_mean,3.768629,3.723143,3.790651,3.746914
review_rating_med,3.800000,3.700000,3.800000,3.800000
review_rating_75th,4.400000,4.300000,4.400000,4.300000
review_rating_max,5.000000,5.000000,5.000000,5.000000
review_rating_std,0.715317,0.717671,0.719843,0.702598
review_rating_var,0.511679,0.515052,0.518173,0.493644
review_rating_sem,0.020314,0.017220,0.029412,0.039033


In [ ]:
# simple crosstab counts
pd.crosstab(df['Frequency of Purchases'], df['Age_Group'])

Age_Group,age_1,age_2,age_3,age_4
Frequency of Purchases,,,,
Annually,132,152,129,159
Bi-Weekly,166,116,139,126
Every 3 Months,167,141,140,136
Fortnightly,147,132,138,125
Monthly,135,143,151,124
Quarterly,154,117,148,144
Weekly,127,141,141,130


In [ ]:
# crosstab with normalized counts by column
pd.crosstab(df['Frequency of Purchases'], df['Age_Group'], margins=True, normalize='columns').round(2)

Age_Group,age_1,age_2,age_3,age_4,All
Frequency of Purchases,,,,,
Annually,0.13,0.16,0.13,0.17,0.15
Bi-Weekly,0.16,0.12,0.14,0.13,0.14
Every 3 Months,0.16,0.15,0.14,0.14,0.15
Fortnightly,0.14,0.14,0.14,0.13,0.14
Monthly,0.13,0.15,0.15,0.13,0.14
Quarterly,0.15,0.12,0.15,0.15,0.14
Weekly,0.12,0.15,0.14,0.14,0.14


In [ ]:
df_pur_sub_status = df.pivot_table(index=['Frequency of Purchases'], columns=['Subscription Status'],values=['Previous Purchases'], aggfunc=['mean'])
df_pur_sub_status

mean           
                       Previous Purchases           
Subscription Status                    No        Yes
Frequency of Purchases                              
Annually                        24.543689  24.606250
Bi-Weekly                       24.002457  27.071429
Every 3 Months                  24.337209  26.701299
Fortnightly                     24.969152  26.039216
Monthly                         25.059406  25.872483
Quarterly                       27.236407  25.700000
Weekly                          25.392670  26.694268

In [ ]:
# stack() example
df_pur_sub_status.stack()

/tmp/ipython-input-22-1692869919.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_pur_sub_status.stack()


mean
                                           Previous Purchases
Frequency of Purchases Subscription Status                   
Annually               No                           24.543689
                       Yes                          24.606250
Bi-Weekly              No                           24.002457
                       Yes                          27.071429
Every 3 Months         No                           24.337209
                       Yes                          26.701299
Fortnightly            No                           24.969152
                       Yes                          26.039216
Monthly                No                           25.059406
                       Yes                          25.872483
Quarterly              No                           27.236407
                       Yes                          25.700000
Weekly                 No                           25.392670
                       Yes                          26.694268

In [ ]:
# multiple column groupby, create MultiIndex, multi-level index
df.groupby(['Frequency of Purchases', 'Subscription Status'])['Previous Purchases'].mean()

Frequency of Purchases  Subscription Status
Annually                No                     24.543689
                        Yes                    24.606250
Bi-Weekly               No                     24.002457
                        Yes                    27.071429
Every 3 Months          No                     24.337209
                        Yes                    26.701299
Fortnightly             No                     24.969152
                        Yes                    26.039216
Monthly                 No                     25.059406
                        Yes                    25.872483
Quarterly               No                     27.236407
                        Yes                    25.700000
Weekly                  No                     25.392670
                        Yes                    26.694268
Name: Previous Purchases, dtype: float64

In [ ]:
# df.unstack() example
df.groupby(['Frequency of Purchases', 'Subscription Status'])['Previous Purchases'].mean().unstack()

Subscription Status,No,Yes
Frequency of Purchases,,
Annually,24.543689,24.606250
Bi-Weekly,24.002457,27.071429
Every 3 Months,24.337209,26.701299
Fortnightly,24.969152,26.039216
Monthly,25.059406,25.872483
Quarterly,27.236407,25.700000
Weekly,25.392670,26.694268


---